In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import networkx as nx
import asyncio
import aiohttp
import json


In [7]:
def fetch_names(url):
    # Create an empty string to store the concatenated content
    all_content = ""
    
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
        # Check if the request was successful
    if response.status_code == 200:
            # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
    
            # Find the element with class 'category-page__members'
        category_members = soup.find('div', {'class': 'category-page__members'})
    
            # Extract the content within the 'category-page__members' div
        if category_members:
            content = category_members.get_text()
                # Append the content to the all_content string
            all_content += content
        else:
            print("Div with class 'category-page__members' not found on the page.")
    else:
            print("Failed to retrieve the page. Status code:", response.status_code)
    
    return all_content


def clean_names(string):
    
    cleaned_string = re.sub(r'.\t', '', string.replace('\n', ' ')).replace('\t', ' ')
    
    # Split the input string by two or more whitespace characters using regular expression
    name_list = re.split(r'\s{2,}', cleaned_string)
    
    # Filter out any empty strings
    name_list = [name.strip() for name in name_list if name.strip()]
    
    clean_list = []
    for name in name_list:
        name=name.replace(' ', '_')
        clean_list.append(name)

    return clean_list


In [8]:
# Base URL for Narutopedia character category
url_villages_list = 'https://naruto.fandom.com/wiki/Category:Villages'

villages_string = fetch_names(url_villages_list)
villages_list = clean_names(villages_string)


In [9]:
villages_list

['Amegakure',
 'Bamboo_Village',
 'Ceramic_Village',
 'Curtain_Village',
 'Daidai_Village',
 'Genjutsu_Tree_Village',
 'Hachō_Village',
 'Hoshigakure',
 'Howling_Wolf_Village',
 'Inaho_Village',
 'Ishigakure',
 'Iwagakure',
 'Jōmae_Village',
 'Kagerō_Village',
 'Kirigakure',
 'Kisaragi_Village',
 'Konohagakure',
 'Kumogakure',
 'Kusagakure',
 'Mount_Katsuragi',
 'Mount_Myōboku',
 'Nadeshiko_Village',
 'Otogakure',
 'Ryūchi_Cave',
 'Shangri-la',
 'Shikkotsu_Forest',
 'Shimogakure',
 'Sugi_Village',
 'Sunagakure',
 'Takigakure',
 'Takumi_Village',
 'Tanigakure',
 'Tonika_Village',
 'Tsuchigumo_Village',
 'Tsukigakure',
 'Uzushiogakure',
 'Yugakure',
 'Yukigakure',
 'Yumegakure']